<a href="https://colab.research.google.com/github/yisi22/Real-Estate-Investment/blob/main/Duxton_Area_Rent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.api as sm

In [2]:
df = pd.read_excel('/content/Shophouse Rental Updated Till Jan 2024.xlsx', parse_dates= True)

In [3]:
rental = df[['Floor/Unit', 'Location', 'Type', 'Lease Start', 'Rent psf']]

In [4]:
# get Lease start day to end of the month
rental['Lease Start'] = rental['Lease Start'] + pd.offsets.MonthEnd(0)
# get the level
rental['Level'] = rental['Floor/Unit'].str.split(' ', expand = True)[0].str.replace('L','').str.replace('#', '').str.replace('0','').str.split('-',expand = True)[0].str.replace('GF', '1').str.replace('Ground', '1')
mask = ['Whole', '1&2&3', 'A&B']
rental = rental[~(rental['Level'].isin(mask)) ]
rental['Level'] = rental['Level'].str[0].astype('int')
# get the usage
rental['Usage'] = np.where(rental['Type'] == 'Office',0,1)

<ipython-input-4-cc95a641acf5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental['Lease Start'] = rental['Lease Start'] + pd.offsets.MonthEnd(0)
<ipython-input-4-cc95a641acf5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental['Level'] = rental['Floor/Unit'].str.split(' ', expand = True)[0].str.replace('L','').str.replace('#', '').str.replace('0','').str.split('-',expand = True)[0].str.replace('GF', '1').str.replace('Ground', '1')
<ipython-input-4-cc95a641acf5>:7: SettingWithCopyWarning: 
A value

In [9]:
rental.groupby('Location').count()

,Floor/Unit,Type,Lease Start,Rent psf,Level,Usage
Location,,,,,,
Craig,8,8,8,8,8,8
Duxton,71,71,71,71,71,71
Duxton Hill,22,22,22,22,22,22
Tras,42,42,42,42,42,42


In [8]:
rental= rental[rental['Location'] != 'Neil']

In [10]:
# get the dummy variables
final_data = pd.get_dummies(rental, columns = ['Location'],drop_first = True)
# Time difference
final_data['Month Difference'] = np.round(((final_data['Lease Start'] - pd.to_datetime('2020-01-01')).dt.days/30))

In [11]:
final_data = final_data.drop(columns = ['Floor/Unit', 'Type'])

In [12]:
final_data

,Lease Start,Rent psf,Level,Usage,Location_Duxton,Location_Duxton Hill,Location_Tras,Month Difference
0,2020-02-29,7.179487,2,0,1,0,0,2.0
1,2020-02-29,7.179487,3,0,1,0,0,2.0
2,2020-03-31,6.875000,3,0,0,1,0,3.0
3,2020-03-31,6.953224,3,0,0,0,1,3.0
4,2020-04-30,6.909091,2,0,0,1,0,4.0
...,...,...,...,...,...,...,...,...
153,2023-12-31,8.250000,2,0,0,0,1,49.0
154,2023-12-31,7.888889,2,1,1,0,0,49.0
155,2024-01-31,6.833333,2,0,1,0,0,50.0
157,2024-02-29,11.944444,1,1,0,0,1,51.0


### Get the Inflation data

In [13]:
inflation = pd.read_excel('/content/Inflation Rate.xlsx', parse_dates = True)

In [14]:
# set inflation as year
inflation['Year'] = pd.to_datetime(inflation['Year'], format='%Y').dt.to_period('Y').dt.to_timestamp('Y')
inflation = inflation.set_index('Year')
# Interpolate the inflation data
inflat = inflation.resample('M').interpolate()
inflat = inflat[['Inflation']]
inflat = inflat.reset_index()

In [15]:
inflat

,Year,Inflation
0,2010-12-31,0.028200
1,2011-01-31,0.030225
2,2011-02-28,0.032250
3,2011-03-31,0.034275
4,2011-04-30,0.036300
...,...,...
152,2023-08-31,0.052400
153,2023-09-30,0.051300
154,2023-10-31,0.050200
155,2023-11-30,0.049100


### Get the market index data

In [16]:
Market_rent = pd.read_csv('/content/Market Rent Index and Vacancy Rate.csv', parse_dates = True)

In [17]:
# Convert the quarterly data
Market_rent['Year'] = Market_rent['Quarter'].str.slice(0, 4)
Market_rent['Quarter'] = Market_rent['Quarter'].str.slice(4)

# Convert 'Quarter' column to datetime objects with the end of each quarter
quarter_mapping = {'Q1': '-03-31', 'Q2': '-06-30', 'Q3': '-09-30', 'Q4': '-12-31'}
Market_rent['Quarter'] = pd.to_datetime(Market_rent['Year'] + Market_rent['Quarter'].map(quarter_mapping))

# Drop 'Year' column as it's no longer needed
Market_rent.drop(columns=['Year'], inplace=True)

# Interpolate the market index
Market_rent = Market_rent.set_index('Quarter')
Market_index = Market_rent.resample('M').interpolate()
Market_index = Market_index.reset_index()

In [18]:
Market_index

,Quarter,Rental Index of Private Sector Retail Space in Central Area (INDEX),Vacancy Rate of Private & Public Sector Retail Space (Whole Island) (per cent)
0,2018-03-31,101.200000,7.500000
1,2018-04-30,100.866667,7.433333
2,2018-05-31,100.533333,7.366667
3,2018-06-30,100.200000,7.300000
4,2018-07-31,99.800000,7.400000
...,...,...,...
65,2023-08-31,81.433333,7.300000
66,2023-09-30,81.600000,7.200000
67,2023-10-31,81.566667,6.966667
68,2023-11-30,81.533333,6.733333


### Merge the data together

In [19]:
final_data = final_data.merge(inflat, left_on ='Lease Start', right_on = 'Year', how = 'left')
final_data = final_data.merge(Market_index, left_on = 'Lease Start', right_on = 'Quarter', how = 'left')
final_data = final_data.dropna()
final_data = final_data.drop(columns = ['Year','Quarter'])

In [20]:
final_data

,Lease Start,Rent psf,Level,Usage,Location_Duxton,Location_Duxton Hill,Location_Tras,Month Difference,Inflation,Rental Index of Private Sector Retail Space in Central Area (INDEX),Vacancy Rate of Private & Public Sector Retail Space (Whole Island) (per cent)
0,2020-02-29,7.179487,2,0,1,0,0,2.0,0.004450,101.600000,7.833333
1,2020-02-29,7.179487,3,0,1,0,0,2.0,0.004450,101.600000,7.833333
2,2020-03-31,6.875000,3,0,0,1,0,3.0,0.003825,101.100000,8.000000
3,2020-03-31,6.953224,3,0,0,0,1,3.0,0.003825,101.100000,8.000000
4,2020-04-30,6.909091,2,0,0,1,0,4.0,0.003200,99.433333,8.533333
...,...,...,...,...,...,...,...,...,...,...,...
135,2023-10-31,6.933333,1,1,1,0,0,47.0,0.050200,81.566667,6.966667
136,2023-11-30,6.909091,4,0,0,1,0,48.0,0.049100,81.533333,6.733333
137,2023-12-31,6.200000,4,0,1,0,0,49.0,0.048000,81.500000,6.500000
138,2023-12-31,8.250000,2,0,0,0,1,49.0,0.048000,81.500000,6.500000


### Linear Regression

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score

In [22]:
X_set = final_data.drop(columns =['Lease Start', 'Rent psf'])
# scaler = StandardScaler()
# X_set_2 = X_set.iloc[:,-4:]
# X_set_scaled = scaler.fit_transform(X_set_2)
# X_set_scaled_df = pd.DataFrame(X_set_scaled, columns = final_data.columns[-4:])
# X_set_1 = X_set.iloc[:,:-4]
# X_set = pd.concat([X_set_1,X_set_scaled_df], axis =1)
X_set = sm.add_constant(X_set)
X = np.asarray(X_set)
y = np.asarray(final_data['Rent psf'])
model = sm.OLS(y,X).fit()

In [23]:
# measure the performance
y_pred = model.predict(X)
mse = mean_squared_error(y,y_pred)
r_squared = r2_score(y,y_pred)
print('Mean Squared Error: ', mse, 'R square: ', r_squared)

Mean Squared Error:  2.3454906252986567 R square:  0.47188363332673167


In [35]:
# set the input value
input_2 = [50,0.05,81.5,6.5]
# input_2 = np.array(input_2).reshape(-1,1)
# input_2_scaled = scaler.fit_transform(input_2)
# input_2 = input_2_scaled.flatten().tolist()
input_1 = [1,1,1,1,0,0]
input = input_1 + input_2

In [36]:
predicted_rent = model.predict(input)
print('The predicted rent is', predicted_rent)

The predicted rent is [10.08211927]


In [37]:
model.params

array([ 1.22934011e+01, -1.04592487e+00,  1.17548012e+00,  3.38382473e-02,
        1.27953267e+00,  3.06799646e-01,  1.37760359e-02,  8.14482901e-01,
       -3.96157716e-03, -4.27897339e-01])

### Machine Learning


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state= 0)

In [ ]:
X

array([[ 1.        ,  2.        ,  0.        , ..., -1.4255827 ,
         4.37372835, -0.29726761],
       [ 1.        ,  3.        ,  0.        , ..., -1.4255827 ,
         4.37372835, -0.29726761],
       [ 1.        ,  3.        ,  0.        , ..., -1.45649752,
         4.25338117,  0.00675608],
       ...,
       [ 1.        ,  4.        ,  0.        , ...,  0.7285622 ,
        -0.46422808, -2.72945716],
       [ 1.        ,  2.        ,  0.        , ...,  0.7285622 ,
        -0.46422808, -2.72945716],
       [ 1.        ,  2.        ,  1.        , ...,  0.7285622 ,
        -0.46422808, -2.72945716]])

In [ ]:
# Linear Regression
lr=LinearRegression().fit(X_train,y_train)
Score_linear_train = lr.score(X_train,y_train)
Score_linear_test = lr.score(X_test,y_test)
print('Training R score of linear regression', Score_linear_train)
print('Testing R score of linear regression', Score_linear_test)

Training R score of linear regression 0.5166844507615478
Testing R score of linear regression 0.02388907682084951


In [ ]:
# Ridge
Ridge_ML=Ridge(alpha=0.2).fit(X_train,y_train)
Score_Ridge_train = Ridge_ML.score(X_train,y_train)
Score_Ridge_test = Ridge_ML.score(X_test,y_test)
print('Training R score of Ridge', Score_Ridge_train)
print('Testing R score of Ridge', Score_Ridge_test)

Training R score of Ridge 0.5161794253269617
Testing R score of Ridge 0.05104098262890766


In [ ]:
# Lasso
Lasso_ML=Lasso(alpha=0.2).fit(X_train,y_train)
Score_Lasso_train = Lasso_ML.score(X_train,y_train)
Score_Lasso_test = Lasso_ML.score(X_test,y_test)
print('Training R score of Lasso', Score_Lasso_train)
print('Testing R score of Lasso', Score_Lasso_test)

Training R score of Lasso 0.3875502881802997
Testing R score of Lasso 0.26190450013460254


In [ ]:
# SVM (poly)
svm1 = svm.SVR(kernel = 'poly', C = 1.0)
svm1.fit(X_train, y_train)
Score_SVM1_train = svm1.score(X_train,y_train)
Score_SVM1_test = svm1.score(X_test,y_test)
print('Training R score of SVM (poly)', Score_SVM1_train)
print('Testing R score of SVM (poly)', Score_SVM1_test)
# SVM (Gaussian)
svm2 = svm.SVR(kernel = 'rbf', C = 1.0, gamma = 'scale')
svm2.fit(X_train, y_train)
Score_SVM2_train = svm2.score(X_train,y_train)
Score_SVM2_test = svm2.score(X_test,y_test)
print('Training R score of SVM (Gaussian)', Score_SVM2_train)
print('Testing R score of SVM (Gaussian)', Score_SVM2_test)

Training R score of SVM (poly) 0.3518033393861777
Testing R score of SVM (poly) 0.18406349336688943
Training R score of SVM (Gaussian) 0.5140510107973923
Testing R score of SVM (Gaussian) 0.3203270971694946


In [ ]:
input = np.array(input).reshape(-1,1).transpose()
prediction_SVM = svm2.predict(input)

In [ ]:
print('SVM:', prediction_SVM)

SVM: [8.85664012]
